In [1]:
import os
import sys
import urllib.request
import pandas as pd
import datetime
import pymysql
import re
import json

In [2]:
client_id = "YOUR_CLIENT_ID"
client_secret = "YOUR_CLIENT_SECRET"

url = "https://openapi.naver.com/v1/datalab/search";
body = "{\"startDate\":\"2016-01-01\",\"endDate\":\"2020-09-30\",\"timeUnit\":\"month\",\"keywordGroups\":[{\"groupName\":\"경제\",\"keywords\":[\"경제\"]}]}";

request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
request.add_header("Content-Type","application/json")
response = urllib.request.urlopen(request, data=body.encode("utf-8"))
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    scrapped = response_body.decode('utf-8')
else:
    print("Error Code:" + rescode)

In [3]:
jsonResult = json.loads(scrapped)
print(jsonResult['results'][0]['data'])

[{'period': '2016-01-01', 'ratio': 35.97307}, {'period': '2016-02-01', 'ratio': 32.22267}, {'period': '2016-03-01', 'ratio': 41.7722}, {'period': '2016-04-01', 'ratio': 35.81074}, {'period': '2016-05-01', 'ratio': 38.99718}, {'period': '2016-06-01', 'ratio': 39.79344}, {'period': '2016-07-01', 'ratio': 29.43925}, {'period': '2016-08-01', 'ratio': 29.45465}, {'period': '2016-09-01', 'ratio': 31.11294}, {'period': '2016-10-01', 'ratio': 34.91652}, {'period': '2016-11-01', 'ratio': 37.87626}, {'period': '2016-12-01', 'ratio': 33.00214}, {'period': '2017-01-01', 'ratio': 28.66538}, {'period': '2017-02-01', 'ratio': 24.89539}, {'period': '2017-03-01', 'ratio': 38.44022}, {'period': '2017-04-01', 'ratio': 31.02198}, {'period': '2017-05-01', 'ratio': 37.16396}, {'period': '2017-06-01', 'ratio': 34.34556}, {'period': '2017-07-01', 'ratio': 27.78936}, {'period': '2017-08-01', 'ratio': 30.0368}, {'period': '2017-09-01', 'ratio': 35.97307}, {'period': '2017-10-01', 'ratio': 34.61145}, {'period': 

In [4]:
def extract_year_and_month(peroid):
    year = peroid.split('-')[0]
    month = peroid.split('-')[1]
    return year, month

# DB 연동 및 table 생성

In [5]:
conn = pymysql.connect(host = "127.0.0.1", user = [USER], passwd = [PASSWORD], db = [DATABASE])
cur = conn.cursor()

In [6]:
"""
검색어 트랜드 정보 저장할 table 생성
"""

create_table_query = """
CREATE TABLE IF NOT EXISTS naver_counting(
    id BIGINT(7) NOT NULL AUTO_INCREMENT,
    year bigint(4) NOT NULL,
    month bigint(2) NOT NULL,
    cnt double,
    primary key(id) )
    charset=utf8mb4;
"""
cur.execute(create_table_query)

0

In [7]:
cur.execute("desc news_counting")
table = cur.fetchall()
pd.DataFrame(list(table))

,0,1,2,3,4,5
0,id,bigint(7),NO,PRI,None,auto_increment
1,year,bigint(4),NO,,None,
2,month,bigint(2),NO,,None,
3,cnt,bigint(100),YES,,None,


In [8]:
"""
db table에 저장
"""

for result in jsonResult['results'][0]['data']:
    query = """ 
    Insert into naver_counting (year,month, cnt) values (%d, %d, %f) ;
    """
    year, month = extract_year_and_month(result["period"])
    naver_cnt = result["ratio"]

    mystring = (query % (int(year), int(month), float(naver_cnt)))
    print(mystring)
    cur.execute(mystring)

 
    Insert into naver_counting (year,month, cnt) values (2016, 1, 35.973070) ;
    
 
    Insert into naver_counting (year,month, cnt) values (2016, 2, 32.222670) ;
    
 
    Insert into naver_counting (year,month, cnt) values (2016, 3, 41.772200) ;
    
 
    Insert into naver_counting (year,month, cnt) values (2016, 4, 35.810740) ;
    
 
    Insert into naver_counting (year,month, cnt) values (2016, 5, 38.997180) ;
    
 
    Insert into naver_counting (year,month, cnt) values (2016, 6, 39.793440) ;
    
 
    Insert into naver_counting (year,month, cnt) values (2016, 7, 29.439250) ;
    
 
    Insert into naver_counting (year,month, cnt) values (2016, 8, 29.454650) ;
    
 
    Insert into naver_counting (year,month, cnt) values (2016, 9, 31.112940) ;
    
 
    Insert into naver_counting (year,month, cnt) values (2016, 10, 34.916520) ;
    
 
    Insert into naver_counting (year,month, cnt) values (2016, 11, 37.876260) ;
    
 
    Insert into naver_counting (year,month, cnt) v

In [9]:
conn.commit()
cur.close()
conn.close()